In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SAM 3 エージェント (SAM 3 Agent)

このノートブックでは、MLLM (Multi-modal Large Language Model) が SAM 3 をツールとして使用する例、すなわち「SAM 3 エージェント」について解説します。
これにより、「青いベストを着ている一番左の子供 (the leftmost child wearing blue vest)」のような、より複雑なテキストクエリに基づいたセグメンテーションが可能になります。

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。
これにより、必要なライブラリのインストールと、データの読み込み準備が行われます。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

## 環境設定 (Env Setup)

まず、リポジトリの [インストール手順](https://github.com/facebookresearch/sam3?tab=readme-ov-file#installation) に従って、環境に `sam3` をインストールしてください。

In [ ]:
import torch
# Ampere GPU向けにtfloat32を有効化
# https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# ノートブック全体でbfloat16を使用。GPUがサポートしていない場合はfloat16を試してください
torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

# ノートブック全体で推論モードを使用。勾配が必要な場合は無効にしてください
torch.inference_mode().__enter__()

In [ ]:
import os

SAM3_ROOT = os.path.dirname(os.getcwd())
os.chdir(SAM3_ROOT)

# 使用するGPUを設定 - このデモの目的には単一のGPUで十分です
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
_ = os.system("nvidia-smi")

## SAM3モデルの構築 (Build SAM3 Model)

SAM3の画像モデルとプロセッサを初期化します。

In [ ]:
import sam3
from sam3 import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor

sam3_root = os.path.join(os.path.dirname(sam3.__file__), "..")
bpe_path = f"{sam3_root}/assets/bpe_simple_vocab_16e6.txt.gz"
model = build_sam3_image_model(bpe_path=bpe_path)
processor = Sam3Processor(model, confidence_threshold=0.5)

## LLMの設定 (LLM Setup)

使用するMLLMを設定します。自分のマシンから起動するvLLMによって提供されるモデル、または外部API経由で提供されるモデルのいずれかを選択できます。
vLLMモデルを使用したい場合は、以下の手順も参照してください。

In [ ]:
LLM_CONFIGS = {
    # vLLMで提供されるモデル
    "qwen3_vl_8b_thinking": {
        "provider": "vllm",
        "model": "Qwen/Qwen3-VL-8B-Thinking",
    }, 
    # 外部API経由で提供されるモデル
    # 必要に応じて追加してください
}

model = "qwen3_vl_8b_thinking"
LLM_API_KEY = "DUMMY_API_KEY"

llm_config = LLM_CONFIGS[model]
llm_config["api_key"] = LLM_API_KEY
llm_config["name"] = model

# APIエンドポイントの設定
if llm_config["provider"] == "vllm":
    LLM_SERVER_URL = "http://0.0.0.0:8001/v1"  # 必要に応じてvLLMサーバーのアドレスに置き換えてください
else:
    LLM_SERVER_URL = llm_config["base_url"]

### vLLMサーバーのセットアップ (Setup vLLM server)

この手順は、vLLMによって提供されるモデルを使用している場合にのみ必要です。GeminiやGPTなどのAPIを使用してLLMを呼び出す場合は、この手順をスキップしてください。

* vLLMをインストールします (依存関係の競合を避けるため、SAM 3とは別のconda環境にインストールしてください)。
  ```bash
    conda create -n vllm python=3.12
    pip install vllm --extra-index-url https://download.pytorch.org/whl/cu128
  ```
* このノートブックと同じマシン上でvLLMサーバーを起動します。
  ```bash
    # qwen 3 VL 8B thinking
    vllm serve Qwen/Qwen3-VL-8B-Thinking --tensor-parallel-size 4 --allowed-local-media-path / --enforce-eager --port 8001
  ```

## SAM3エージェント推論の実行 (Run SAM3 Agent Inference)

画像とプロンプトを指定して、SAM3エージェントによる推論を実行します。

In [ ]:
from functools import partial
from IPython.display import display, Image
from sam3.agent.client_llm import send_generate_request as send_generate_request_orig
from sam3.agent.client_sam3 import call_sam_service as call_sam_service_orig
from sam3.agent.inference import run_single_image_inference

In [ ]:
# 入力引数の準備と単一画像推論の実行
image = "assets/images/test_image.jpg"
prompt = "the leftmost child wearing blue vest"
image = os.path.abspath(image)
send_generate_request = partial(send_generate_request_orig, server_url=LLM_SERVER_URL, model=llm_config["model"], api_key=llm_config["api_key"])
call_sam_service = partial(call_sam_service_orig, sam3_processor=processor)
output_image_path = run_single_image_inference(
    image, prompt, llm_config, send_generate_request, call_sam_service, 
    debug=True, output_dir="agent_output"
)

# 出力の表示
if output_image_path is not None:
    display(Image(filename=output_image_path))